In [2]:
!pip freeze > requirements.txt

In [8]:
from konlpy.tag import Kkma, Komoran # 형태소 분석기 모듈(Hannanum, Kkma, Komoran, Okt)을 탑재
from tqdm.notebook import tqdm
import pandas as pd

#### change csv to parquet

In [15]:
for year in tqdm(range(1980, 2000)):
    df = pd.read_csv(f'./data/chosun_{year}.csv')

    df.to_parquet(f"./data/chosun_{year}.parquet",
                  engine="pyarrow", compression="gzip")

  0%|          | 0/20 [00:00<?, ?it/s]

#### load data

In [6]:
df = pd.read_parquet('./data/chosun_1980.parquet')

TypeError: read_table() got an unexpected keyword argument 'encoding_errors'

In [7]:
df = pd.read_csv(f'./data/chosun_1980.csv', encoding='utf-8', encoding_errors='ignore')

In [ ]:
data = df.loc[:, 'body_archaic_hangul']
N = len(data)
d = 2
r = []
for i in range(1, d+1):
    

#### analysis

In [9]:
kom = Komoran()

data = df.loc[:, 'body_archaic_hangul']

r = []
for text in tqdm(data):
    try:
        pos = kom.pos(text)
    except Exception as error:
        print(error)
        print(text)
    for word, tag in pos:
        if tag.startswith('N'):
            r.append((word, tag))

  0%|          | 0/35907 [00:00<?, ?it/s]

'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
조선일보(朝鮮日報) 신춘문예(新春文藝) 당선소설집(小說集) 출간
「신인(新人)」들의 문제작(問題作)‥34편 정선(精選)
 조선일보사(朝鮮日報社)는 창간60주년기념 출판의 하나로 조선일보문학선서(朝鮮日報文學選書) 제1,2,3집「조선일보(朝鮮日報) 신춘문예당선소설집(新春文藝當選小說集)」전3권을 펴냅니다.한국 신문 최초로 신춘문예 현상공모를시작한 조선일보(朝鮮日報)가 1929년부터 79년까지 약 50년간의신춘문예당(新春文藝當)—입선(入選)소설 70여편중에서 34편을 정선해 모은 소설작품집입니다.여기 수록된 소설들은 그때그때 치열한 경쟁을 통해서 첫째를 차지한,해마다 새로왔던 참신한 신인(新人)들의 문제작(問題作)으로서 한국 현대 단편소설의 발전단계를 몸으로 실중해주는 작품들이기도 합니다.또한 각편마다 그 시대의 사회상(社會相),문학적 특성들이 고루 고루 나타나 있읍니다.30년대 일제치하(日帝治下)에 새로 싹트는 반일(反日)과 근대시민의식(近代市民意識),50~60년대의 전쟁과 전후(戰後)의 궁핍,정치적 혼란,변혁,70년대의 경제성장,안보,안정과 자유(自由)에의 의지와의 갈등들이그때그때 새로운 소설의 시각(視角)을 따라 여실히 펼쳐집니다.애독자 여러분의 성원(聲援)과 편달을 바랍니다.
전(全)3권(卷) 값 5,400원
<전국유명서점(書店)서발매중>
문의처:본사출판판매부(本社出版販賣部) 75-6611~9
(교(交))110번(番)73-1734
<수록작가(収錄作家) 34명>
전춘호(田春湖) 김용송(金龍松) 정순정이영근(鄭順貞李永根) 안𨒿순(安𨒿淳) 현(玄) 홍박영준(鴻朴榮濬) 김유정(金裕貞) 김정한정비석(金廷漢鄭飛石) 김영수(金永壽) 최창대(崔昌大)
(이상제1권·396면·1천9백원)
전광용(全光鏞) 성학원(成鶴園) 최현식이병구(崔玄植李丙求) 양문길(梁文吉) 박순여김문수(朴順女金文洙) 전상국(全商國) 김수남최인호(金秀男崔仁浩) 이세기(李世

KeyboardInterrupt: 

In [12]:
from concurrent import futures
import time

def pos_parse(i, data):
    kkm = Kkma()
    r = []
    for text in tqdm(data, position=i):
        try:
            pos = kkm.pos(text)
        except Exception as error:
            print(error)
            print(text)
        for word, tag in pos:
            if tag.startswith('N'):
                r.append((word, tag))
    return i, r

In [11]:
%%time
with futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(sum, i, 10) for i in range(3)]

[2] 0[0] 0

[1] 0
[0] 1
[1] 1
[2] 1
[0] 2
[1] 2
[2] 2
[0] 3[1] 3
[2] 3

[1] 4
[2] 4
[0] 4
[1] 5
[2] 5
[0] 5
[1] 6
[2] 6
[0] 6
[1] 7[2] 7

[0] 7
[1] 8
[2] 8
[0] 8
[1] 9[0] 9
thread 0 exit

thread 1 exit
[2] 9
thread 2 exit
CPU times: user 3.94 ms, sys: 2.89 ms, total: 6.83 ms
Wall time: 1.03 s


In [5]:
for f in futures.as_completed(results):
    print(f.result())


print('main exits')

(0, 45)
(2, 45)
(1, 45)
main exits


In [16]:
def pos_parse(i, data):
    kkm = Kkma()
    r = []
    for text in tqdm(data, position=i):
        try:
            pos = kkm.pos(text)
        except Exception as error:
            print(error)
            print(text)
        for word, tag in pos:
            if tag.startswith('N'):
                r.append((word, tag))
    return i, r

In [6]:
df = df.loc[:, 'body_archaic_hangul']


In [ ]:
N = len(df)
data = df[:100]

r = pos_parse(0, data)

In [31]:
r = r[1]
res = []
for word, tag in r:
    if not tag.startswith('NNB'):
        res.append((word, tag))

In [32]:
from collections import Counter
Counter(res).most_common(100)

[(('의', 'NNG'), 207),
 (('우리', 'NP'), 191),
 (('이', 'NNG'), 140),
 (('을', 'NNG'), 129),
 (('국민', 'NNG'), 120),
 (('문제', 'NNG'), 120),
 (('때', 'NNG'), 114),
 (('순리', 'NNG'), 113),
 (('사회', 'NNG'), 111),
 (('1', 'NR'), 108),
 (('80', 'NR'), 100),
 (('말', 'NNG'), 99),
 (('일', 'NNM'), 98),
 (('김', 'NNG'), 95),
 (('생각', 'NNG'), 91),
 (('년', 'NNM'), 90),
 (('사람', 'NNG'), 89),
 (('일', 'NNG'), 88),
 (('경제', 'NNG'), 86),
 (('3', 'NR'), 86),
 (('나', 'NP'), 84),
 (('2', 'NR'), 81),
 (('새해', 'NNG'), 79),
 (('년대', 'NNM'), 78),
 (('정치', 'NNG'), 77),
 (('그', 'NP'), 67),
 (('백', 'NR'), 65),
 (('대통령', 'NNG'), 64),
 (('역사', 'NNG'), 64),
 (('정부', 'NNG'), 63),
 (('명', 'NNM'), 60),
 (('앞', 'NNG'), 60),
 (('발전', 'NNG'), 56),
 (('그것', 'NP'), 55),
 (('안정', 'NNG'), 55),
 (('은', 'NNG'), 55),
 (('천', 'NR'), 53),
 (('만', 'NR'), 52),
 (('성장', 'NNG'), 49),
 (('세계', 'NNG'), 49),
 (('월', 'NNM'), 49),
 (('경우', 'NNG'), 48),
 (('5', 'NR'), 47),
 (('4', 'NR'), 47),
 (('현실', 'NNG'), 47),
 (('저', 'NP'), 44),
 (('얘기', 'NNG'

In [33]:
df.head(50)

0     새해 국민(國民)경제 기상(氣象)\n인플레-실업(失業)……\n"먹구름"예보\n전례(...
1     변혁(变革)의길\n①\n개헌(改憲)논의\n흑백론(黑白論) 판치면 시간(時間)낭비\n...
2     정부(政府)—기업—가계(家計) 힘 뭉쳐야\n경제(經濟)단체 노사(勞使)협조‥생활 질...
3     근로자\n경영인(人)이 합리적(合理的)이면 우리도 따라\n 우리나라를 둘러싼 역사의...
4     80연도(年度) 신춘문예(新春文藝) 당선작(當選作) 발표\n사고(社告)\n 1980...
5     대학원(大學院) 정원(定員)늘려\n80학년도(學年度) 신입생(新入生) 3천2백명 더...
6     80연대(年代)「제1막(幕) 1장(場)」을 연다\n ◇1980년의 새 아침이 밝는다...
7     이웃돕기 성금(誠金)\n일신방직(日新紡織) 3백만원\n▲(주(株))일신방직(日新紡織...
8     1월의 메모\n▲1일=신정(新正)▲4일=시무식▲6일=소한(小寒)▲8일=이봉창(李奉昌...
9              주택복권 당첨번호\n<제408회>\n주택복권 당첨번호\n<제408회>\n
10    ◇본사사령(本社辭令)\n편집국부국장(編輯局副局長) 조(曺) 영(永) 서(瑞)\n명(...
11    조선일보(朝鮮日報) 창간(創刊) 60주년 기념(紀念)사업\n조선일보사(朝鮮日報社)\...
12    미래학(未來学) 교수(教授)가 진단하는 한국상(韓国像)\n80연대(年代)를 그린다\...
13    미래소설(未來小説)에 나타난「80년대 여언(予言)」\n"회색(灰色)"짙은 전운(戰雲...
14    설문(設問)으로 엮여본 10제(題)\n80년대 이렇게 된다…꿈\n금강산(金剛山) 관...
15                     ◇조세형(趙世衡)\n<국회의원(國會議員)·신민(新民)>\n
16    □의 여언(予言)"\n현실(現実)과 바램과‥‥\n생각하십니까‥‥베를린장벽(障壁)도 ...
17                                           ◇앤터

In [36]:
kkm = Kkma()

kkm.pos('젊은이반대(反對)의 힘 용납할때 화해점(和解點) 발견')

[('젊은이', 'NNG'),
 ('반대', 'NNG'),
 ('(', 'SS'),
 ('反對', 'OH'),
 (')', 'SS'),
 ('의', 'NNG'),
 ('힘', 'NNG'),
 ('용납', 'NNG'),
 ('하', 'XSV'),
 ('ㄹ', 'ETD'),
 ('때', 'NNG'),
 ('화해', 'NNG'),
 ('점', 'NNG'),
 ('(', 'SS'),
 ('和解點', 'OH'),
 (')', 'SS'),
 ('발견', 'NNG')]

In [45]:
import hanja
from hanja import hangul

In [49]:
# TODO: implement in reverse order
# TODO: return hanja removed string
def hanja_parse(string):
    i = 0
    r = []
    while i < len(string):
        if hanja.is_hanja(string[i]):
            hanja_cnt = 1
            hanja_word = string[i]
            j = i + 1
            while hanja.is_hanja(string[j]):
                hanja_word += string[j]
                hanja_cnt += 1
                j += 1
            hangul_cnt = 0
            hangul_word = ''
            for k in range(i, -1, -1):
                s = string[k]
                if hangul.is_hangul(s):
                    hangul_word += s
                    hangul_cnt += 1

                    if hangul_cnt >= hanja_cnt:
                        break
            r.append((hangul_word[::-1], hanja_word))
            i = j
        else:
            i += 1
    return r

In [64]:
pos_parse('젊은이반대(反對)의 힘 용납할때 화해점(和解點) 발견')

[('젊은이', 'NNG'),
 ('반대', 'NNG'),
 ('의', 'NNG'),
 ('힘', 'NNG'),
 ('용납', 'NNG'),
 ('때', 'NNG'),
 ('화해', 'NNG'),
 ('점', 'NNG'),
 ('발견', 'NNG')]

In [59]:
hanja_parse('젊은이반대(反對)의 힘 용납할때 화해점(和解點) 발견')

[('반대', '反對'), ('화해점', '和解點')]

In [58]:
def pos_parse(string):
    kkm = Kkma()
    r = []
    try:
        pos = kkm.pos(string)
    except Exception as error:
        print(error)
        print(text)
    for word, tag in pos:
        if tag.startswith('N'):
            r.append((word, tag))
    return r

In [7]:
test = df[11]
test *= 100

In [8]:
from konlpy.tag import Kkma, Hannanum, Komoran, Okt
kkm = Kkma()
han = Hannanum()
kom = Komoran()
okt = Okt()

In [9]:
%%timeit
kkm.pos(test)

16.2 s ± 342 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
han.pos(test)

7.5 s ± 79.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
kom.pos(test)

897 ms ± 50.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
okt.pos(test)

1.81 s ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
kom.pos('킹갓제네럴엠페러충무공마제스티하이퍼울트라판타스틱익스트림지니어스화룡정점마스터더엠비션리마스터성골판타스틱엘라스틱레전더리충무공6인궁 윤필립')

[('킹갓제네럴엠페러충무공마제스티하이퍼울트라판타스틱익스트림지니어스화룡정점마스터더엠비션리마스터성골판타스틱엘라스틱레전더리충무공6인궁',
  'NA'),
 ('윤', 'NNP'),
 ('필립', 'NNP')]